# This notebook is for making projections of stellar density

### Can only make projections since it's a particle field instead of baryonic field

In [ ]:
###############################
# Script for making stellar
# density projections
###############################

# imports and parallelism
import yt
from yt.data_objects.particle_filters import add_particle_filter

yt.enable_parallelism()

# make the timeseries
ts = yt.load("/mnt/research/galaxies-REU/sims/claire_isogal_sims/newCGM_h2sf_no-ramp/DD????/DD????")

# loop through all the snapshots in the sim
for ds in ts.piter():
    
    # making sure there are particles to run it
    if ds.particles_exist == False:
        continue
        
    filename = ds.filename.split('/')[-1]
    
    # make the filter and add it
    def formed_star(pfilter, data):
        filter = data['all', 'creation_time'] >0
        return filter

    add_particle_filter('formed_star', function=formed_star, filtered_type='all', requires=['creation_time'])
    
    ds.add_particle_filter("formed_star")
    
    # make cylinders to make projections. Radius may change depending on sim used
    myxcylinder = ds.disk([0.5,0.5,0.5], [1,0,0], (206,'kpc'), (206,'kpc'))
    myzcylinder = ds.disk([0.5,0.5,0.5], [0,0,1], (206,'kpc'), (50,'kpc'))
    
    # make a projection in the x and z direction
    p_x = yt.ProjectionPlot(ds, 'x', ('deposit', 'formed_star_density'), data_source=myxcylinder, 
                       center = [0.5,0.5,0.5], width=(100, 'kpc'))
    p_x.set_unit('formed_star_density', 'Msun/pc**2')
    p_x.annotate_timestamp(text_args={'color':'dimgrey'})
    p_x.set_zlim(('deposit', 'formed_star_density'), zmin=(1e-3, "Msun/pc**2"), zmax=(1e3, "Msun/pc**2"))
    p_x.save('newCGM_noramp_StellarDensity_Projection_x_snapshot'+filename+'.png')
    
    
    p_z = yt.ProjectionPlot(ds, 'z', ('deposit', 'formed_star_density'), data_source=myzcylinder, 
                       center = [0.5,0.5,0.5], width=(100, 'kpc'))
    p_z.set_unit('formed_star_density', 'Msun/pc**2')
    p_z.annotate_timestamp(text_args={'color':'dimgrey'})
    p_z.set_zlim(('deposit', 'formed_star_density'), zmin=(1e-3, "Msun/pc**2"), zmax=(1e3, "Msun/pc**2"))
    p_z.save('newCGM_noramp_StellarDensity_Projection_z_snapshot'+filename+'.png')

## This section is just for new/young stars

In [ ]:
#######################################
# Script for making the projections
# of young stars for KS law and 
# general knowledge
#######################################

import yt
import np
from yt.data_objects.particle_filters import add_particle_filter

yt.enable_parallelism()

# make the filter for formed stars
def formed_star(pfilter, data):
    filter = data['all', 'creation_time'] >0
    return filter

# make the filter for new/young stars
def new_stars(pfilter, data):
    age = data.ds.current_time - data[pfilter.filtered_type, "creation_time"]
    filter = np.logical_and(age.in_units("Myr") <= 50, age >= 0)
    return filter

# add the filters
add_particle_filter("formed_star", function=formed_star, filtered_type='all', requires=["creation_time"])

add_particle_filter("new_stars", function=new_stars, filtered_type="formed_star", requires=["creation_time"])

# need this data for each snapshot of the simulation, so need to make a timeseries
ts = yt.load("/mnt/research/galaxies-REU/sims/claire_isogal_sims/newCGM_h2sf/DD????/DD????")

for ds in ts.piter():
    
    # making sure there are particles to run it
    if ds.particles_exist == False:
        continue
        
    filename = ds.filename.split('/')[-1]
    myzcylinder = ds.disk([0.5,0.5,0.5], [0,0,1], (206,'kpc'), (50,'kpc'))
    ds.add_particle_filter("new_stars")
    
    # make the projections
    p_stars = yt.ProjectionPlot(ds, 'z', ('deposit', 'new_stars_density'), data_source=myzcylinder, center=[0.5,0.5,0.5], width=(50, 'kpc'))
    p_stars.set_unit('new_stars_density', 'Msun/pc**2')
    p_stars.set_zlim(('deposit', 'new_stars_density'), zmin=1e-2, zmax=1e3)
    p_stars.annotate_timestamp(text_args={'color':'dimgrey'})
    p_stars.save('newCGM_newstarsdens_proj_z_snapshot_'+filename+'.png')